<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/Alpaca_%2B_Llama_3_8b_full_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

要在免费的Tesla T4 Google Colab实例上运行此代码，请按“*Runtime*”，然后按“*Run all*”！
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> 如果需要帮助，请加入Discord，如果可以请支持我们！
</div>

要在您自己的计算机上安装Unsloth，请按照我们Github页面上的安装说明[here](https://github.com/unslothai/unsloth#installation-instructions---conda)进行操作。

您将学习如何进行[数据准备](#Data)，如何[训练](#Train)，如何[运行模型](#Inference)，以及[如何保存](#Save)（例如用于Llama.cpp）。

**[新] Llama-3 8b经过疯狂地训练了15000亿个标记！ Llama-2是2000亿个标记。**

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

* 我们支持Llama、Mistral、CodeLlama、TinyLlama、Vicuna、Open Hermes等等。
* 以及Yi、Qwen（[llamafied](https://huggingface.co/models?sort=trending&search=qwen+llama)）、Deepseek，所有Llama、Mistral派生的架构。
* 我们支持16位LoRA或4位QLoRA。两者都比原来快2倍。
* `max_seq_length`可以设置为任何值，因为我们通过[kaiokendev的](https://kaiokendev.github.io/til)方法进行自动的RoPE缩放。
* [**新功能**] 通过[PR 26037](https://github.com/huggingface/transformers/pull/26037)，我们支持下载4位模型**快4倍**！[我们的repo](https://huggingface.co/unsloth)有Llama、Mistral的4位模型。

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096*2 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
] # More models at https://huggingface.co/unsloth

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
print(tokenizer)
print(model)

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
unsloth/llama-3-8b-bnb-4bit does not have a padding or unknown token!
Will use the EOS token of id 128001 as padding.


PreTrainedTokenizerFast(name_or_path='unsloth/llama-3-8b-bnb-4bit', vocab_size=128000, model_max_length=8192, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>', 'pad_token': '<|end_of_text|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|reserved_special_token_2|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128005: AddedToke

### 微调前的推理


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13\n\n### Explanation:\nThe fibonnaci sequence is a sequence of numbers where each number is the sum of the previous two numbers. The sequence starts with 1, 1, 2, 3, 5, 8. The next number in the sequence is 13, which is the sum of']

还可以使用`TextStreamer`进行连续推理-因此您可以逐个token查看生成token，而不是等待整个时间!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
13

### Explanation:
The fibonnaci sequence is a sequence of numbers where each number is the sum of the previous two numbers. The sequence starts with 1, 1, 2, 3, 5, 8. The next number in the sequence is 13, which is the sum of the previous two numbers, 8 and 5.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
21

### Explanation:
The fibonnaci sequence is a sequence of numbers where each number is the


我们现在添加了LoRA适配器，所以我们只需要更新所有参数的1%到10% !

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
print(model)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=409

In [ ]:
model.print_trainable_parameters()

trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5195983464188562


### 数据准备

我们现在使用来自[yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned)的Alpaca数据集，这是原始[Alpaca数据集](https://crfm.stanford.edu/2023/03/13/alpaca.html)的经过筛选的版本，共有52,000条数据。您可以用自己的数据准备替换此代码部分。

**[注意]** 要仅对完成部分进行训练（忽略用户输入），请阅读TRL的文档[here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only)。

**[注意]** 记得将**EOS_TOKEN**添加到标记化的输出中！！否则会导致无限生成！

如果您想要使用`ChatML`模板进行ShareGPT数据集，请尝试我们的对话[notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)。

对于像写小说一样的文本补全，可以尝试这个[notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)。

In [ ]:

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

### 训练模型

现在让我们使用Huggingface TRL的`SFTTrainer`！更多文档请查看：[TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer)。我们进行了60步来加快速度，但您可以将`num_train_epochs=1`设置为完整运行，并关闭`max_steps=None`。我们还支持TRL的`DPOTrainer`！

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

In [ ]:
print(trainer.args)

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_lay

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.668 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.814600
2,2.293200
3,1.689500
4,1.952700
5,1.645600
6,1.639600
7,1.217600
8,1.246700
9,1.069300
10,1.174000


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

435.4375 seconds used for training.
7.26 minutes used for training.
Peak reserved memory = 8.982 GB.
Peak reserved memory for training = 3.314 GB.
Peak reserved memory % of max memory = 60.903 %.
Peak reserved memory for training % of max memory = 22.471 %.


#### tensorboard查看训练监控数据

In [ ]:
%load_ext tensorboard
%tensorboard --logdir outputs/runs

### 微调后的推理

让我们运行模型！您可以更改指令和输入 - 将输出留空！

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025']

还可以使用`TextStreamer`进行连续推理-因此您可以逐个token查看生成token，而不是等待整个时间!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 632459


### 保存、加载微调模型

要将最终模型保存为LoRA适配器，可以使用Huggingface的`push_to_hub`进行在线保存，或者使用`save_pretrained`进行本地保存。

**[注意]** 这仅保存LoRA适配器，而不是完整的模型。要保存为16位或GGUF，请向下滚动！

In [ ]:
from google.colab import userdata
token = userdata.get('HF_TOKEN')


In [ ]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = token) # Online saving

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!ls -lh ./lora_model

total 161M
-rw-r--r-- 1 root root  732 Apr 19 15:16 adapter_config.json
-rw-r--r-- 1 root root 161M Apr 19 15:16 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K Apr 19 15:16 README.md


In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=409

现在，如果你想加载我们刚刚为推理保存的LoRA适配器，将`False`设置为`True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


["Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\nOne of the most famous tall towers in Paris is the Eiffel Tower. It is a wrought iron tower located on the Champ de Mars in Paris, France. It was built in 1889 as the entrance to the 1889 World's Fair, and it was designed by the French engineers Gustave Eiff"]

您还可以使用Hugging Face的`AutoModelForPeftCausalLM`。只有在没有安装`unsloth`时才使用它。由于不支持`4bit`模型下载，并且Unsloth的**推理速度快2倍**，因此它可能会非常慢。

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### 保存为float16以供VLLM使用

我们还支持直接保存为`float16`。选择`merged_16bit`以获取float16或`merged_4bit`以获取int4。我们还允许将`lora`适配器作为备用。使用`push_to_hub_merged`将其上传到您的Hugging Face账户！您可以转到 https://huggingface.co/settings/tokens 获取您的个人令牌。

In [ ]:
# Merge to 16bit
if True: model.save_pretrained_merged("model_merged_16bit", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("weege007/llama-3-8b-bnb-4bit-alpaca-merged-16bit", tokenizer, save_method = "merged_16bit", token = token)

# Merge to 4bit
if False: model.save_pretrained_merged("model_merged_4bit", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("weege007/llama-3-8b-bnb-4bit-alpaca-merged-4bit", tokenizer, save_method = "merged_4bit_forced", token = token)

# Just LoRA adapters
if False: model.save_pretrained_merged("model_lora", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("weege007/llama-3-8b-bnb-4bit-alpaca-lora", tokenizer, save_method = "lora", token = token)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.44 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:00<00:00,  1.90s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving model_merged_16bit/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model_merged_16bit/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model_merged_16bit/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model_merged_16bit/pytorch_model-00004-of-00004.bin...
Done.


In [ ]:
!ls -lh ./model_merged_16bit

total 15G
-rw-r--r-- 1 root root  755 Apr 19 12:16 config.json
-rw-r--r-- 1 root root  121 Apr 19 12:16 generation_config.json
-rw-r--r-- 1 root root 4.7G Apr 19 12:18 pytorch_model-00001-of-00004.bin
-rw-r--r-- 1 root root 4.7G Apr 19 12:19 pytorch_model-00002-of-00004.bin
-rw-r--r-- 1 root root 4.6G Apr 19 12:21 pytorch_model-00003-of-00004.bin
-rw-r--r-- 1 root root 1.1G Apr 19 12:21 pytorch_model-00004-of-00004.bin
-rw-r--r-- 1 root root  24K Apr 19 12:21 pytorch_model.bin.index.json
-rw-r--r-- 1 root root  335 Apr 19 12:16 special_tokens_map.json
-rw-r--r-- 1 root root  50K Apr 19 12:16 tokenizer_config.json
-rw-r--r-- 1 root root 8.7M Apr 19 12:16 tokenizer.json


In [ ]:
!ls -lh ./llama-3-8b-bnb-4bit-alpaca-merged-16bit

total 15G
-rw-r--r-- 1 root root  755 Apr 19 12:34 config.json
-rw-r--r-- 1 root root  121 Apr 19 12:34 generation_config.json
-rw-r--r-- 1 root root 4.7G Apr 19 12:35 pytorch_model-00001-of-00004.bin
-rw-r--r-- 1 root root 4.7G Apr 19 12:37 pytorch_model-00002-of-00004.bin
-rw-r--r-- 1 root root 4.6G Apr 19 12:38 pytorch_model-00003-of-00004.bin
-rw-r--r-- 1 root root 1.1G Apr 19 12:39 pytorch_model-00004-of-00004.bin
-rw-r--r-- 1 root root  24K Apr 19 12:39 pytorch_model.bin.index.json
-rw-r--r-- 1 root root  575 Apr 19 12:39 README.md
-rw-r--r-- 1 root root  335 Apr 19 12:34 special_tokens_map.json
-rw-r--r-- 1 root root  50K Apr 19 12:34 tokenizer_config.json
-rw-r--r-- 1 root root 8.7M Apr 19 12:34 tokenizer.json


保存的模型权重文件为pytorch pickle bin文件, 如果想转换成 safetensors 可以直接参考： https://huggingface.co/docs/safetensors/convert-weights

### GGUF / llama.cpp 转换
现在我们原生支持保存到 `GGUF` / `llama.cpp`！我们克隆了 `llama.cpp`，并且默认保存为 `q8_0`。我们允许所有方法，如 `q4_k_m`。使用 `save_pretrained_gguf` 进行本地保存，使用 `push_to_hub_gguf` 进行上传到 HF。

一些支持的量化方法（完整列表请查看我们的[Wiki页面](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)）：
* `q8_0` - 快速转换。资源使用高，但通常可接受。
* `q4_k_m` - 推荐使用。对一半的 attention.wv 和 feed_forward.w2 张量使用 Q6_K，其他使用 Q4_K。
* `q5_k_m` - 推荐使用。对一半的 attention.wv 和 feed_forward.w2 张量使用 Q6_K，其他使用 Q5_K。

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp


Cloning into 'llama.cpp'...
remote: Enumerating objects: 22541, done.
remote: Counting objects: 100% (7127/7127), done.
remote: Compressing objects: 100% (507/507), done.
remote: Total 22541 (delta 6889), reused 6690 (delta 6620), pack-reused 15414
Receiving objects: 100% (22541/22541), 25.42 MiB | 18.24 MiB/s, done.
Resolving deltas: 100% (15979/15979), done.


In [ ]:
!cd llama.cpp && make clean && LLAMA_CUDA=1 make all -j

In [ ]:
!python llama.cpp/convert.py --help

usage: convert.py [-h] [--dump] [--dump-single] [--vocab-only] [--no-vocab]
                  [--outtype {f32,f16,q8_0}] [--vocab-dir VOCAB_DIR] [--vocab-type VOCAB_TYPE]
                  [--outfile OUTFILE] [--ctx CTX] [--concurrency CONCURRENCY] [--big-endian]
                  [--pad-vocab] [--skip-unknown]
                  model

Convert a LLaMA model to a GGML compatible file

positional arguments:
  model                 directory containing model file, or model file itself (*.pth, *.pt, *.bin)

options:
  -h, --help            show this help message and exit
  --dump                don't convert, just show what's in the model
  --dump-single         don't convert, just show what's in a single model file
  --vocab-only          extract only the vocab
  --no-vocab            store model without the vocab
  --outtype {f32,f16,q8_0}
                        output format - note: q8_0 may be very slow (default: f16 or f32 based on
                        input)
  --vocab-dir VOCAB_D

more detail see:
- https://github.com/ggerganov/llama.cpp/issues/6747
- https://github.com/ggerganov/llama.cpp/pull/6745#issuecomment-2064814034

In [ ]:
!python llama.cpp/convert.py model_merged_16bit \
  --outfile model_gguf_q8_0-unsloth.Q8_0.gguf --vocab-type bpe \
  --outtype q8_0 --concurrency 1


Loading model file model_merged_16bit/pytorch_model-00001-of-00004.bin
Loading model file model_merged_16bit/pytorch_model-00001-of-00004.bin
Loading model file model_merged_16bit/pytorch_model-00002-of-00004.bin
Loading model file model_merged_16bit/pytorch_model-00003-of-00004.bin
Loading model file model_merged_16bit/pytorch_model-00004-of-00004.bin
params = Params(n_vocab=128256, n_embd=4096, n_layer=32, n_ctx=8192, n_ff=14336, n_head=32, n_head_kv=8, n_experts=None, n_experts_used=None, f_norm_eps=1e-05, rope_scaling_type=None, f_rope_freq_base=500000.0, f_rope_scale=None, n_orig_ctx=None, rope_finetuned=None, ftype=<GGMLFileType.MostlyQ8_0: 7>, path_model=PosixPath('model_merged_16bit'))
Loaded vocab file PosixPath('model_merged_16bit/tokenizer.json'), type 'bpe'
Vocab info: <BpeVocab with 128000 base tokens and 256 added tokens>
Special vocab info: <SpecialVocab with 280147 merges, special tokens {'bos': 128000, 'eos': 128001, 'pad': 128001}, add special tokens unset>
Permuting 

In [ ]:
!ls -lh model_gguf_q8_0-unsloth.Q8_0.gguf

-rw-r--r-- 1 root root 8.0G Apr 19 17:12 model_gguf_q8_0-unsloth.Q8_0.gguf


In [47]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [48]:
!huggingface-cli upload --repo-type model weege007/llama-3-8b-bnb-4bit-alpaca-merged-16bit model_gguf_q8_0-unsloth.Q8_0.gguf

Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
model_gguf_q8_0-unsloth.Q8_0.gguf: 100% 8.54G/8.54G [03:47<00:00, 37.6MB/s]
https://huggingface.co/weege007/llama-3-8b-bnb-4bit-alpaca-merged-16bit/blob/main/model_gguf_q8_0-unsloth.Q8_0.gguf


In [53]:
!python llama.cpp/convert.py model_merged_16bit \
  --outfile model_gguf-unsloth.f16.gguf --vocab-type bpe \
  --outtype f16 --concurrency 1


Loading model file model_merged_16bit/pytorch_model-00001-of-00004.bin
Loading model file model_merged_16bit/pytorch_model-00001-of-00004.bin
Loading model file model_merged_16bit/pytorch_model-00002-of-00004.bin
Loading model file model_merged_16bit/pytorch_model-00003-of-00004.bin
Loading model file model_merged_16bit/pytorch_model-00004-of-00004.bin
params = Params(n_vocab=128256, n_embd=4096, n_layer=32, n_ctx=8192, n_ff=14336, n_head=32, n_head_kv=8, n_experts=None, n_experts_used=None, f_norm_eps=1e-05, rope_scaling_type=None, f_rope_freq_base=500000.0, f_rope_scale=None, n_orig_ctx=None, rope_finetuned=None, ftype=<GGMLFileType.MostlyF16: 1>, path_model=PosixPath('model_merged_16bit'))
Loaded vocab file PosixPath('model_merged_16bit/tokenizer.json'), type 'bpe'
Vocab info: <BpeVocab with 128000 base tokens and 256 added tokens>
Special vocab info: <SpecialVocab with 280147 merges, special tokens {'bos': 128000, 'eos': 128001, 'pad': 128001}, add special tokens unset>
Permuting l

In [59]:
!ls -lh model_gguf-unsloth.f16.gguf

-rw-r--r-- 1 root root 15G Apr 19 18:07 model_gguf-unsloth.f16.gguf


In [55]:
!huggingface-cli upload --repo-type model weege007/llama-3-8b-bnb-4bit-alpaca-merged-16bit model_gguf-unsloth.f16.gguf

Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
model_gguf-unsloth.f16.gguf: 100% 16.1G/16.1G [07:01<00:00, 38.1MB/s]
https://huggingface.co/weege007/llama-3-8b-bnb-4bit-alpaca-merged-16bit/blob/main/model_gguf-unsloth.f16.gguf


In [60]:
!./llama.cpp/main -ngl 33 -c 0 -e \
  -p '<|start_header_id|>user<|end_header_id|>\n\nWhat is the capital of France?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n' \
  -r '<|eot_id|>' \
  -m model_gguf-unsloth.f16.gguf \
  && echo "The capital of France is Paris."

Log start
main: build = 2698 (637e9a86)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1713550853
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from model_gguf-unsloth.f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:            

In [61]:
!./llama.cpp/quantize model_gguf-unsloth.f16.gguf model_gguf_q4_k_m-unsloth.Q4_k_m.gguf q4_k_m


main: build = 2698 (637e9a86)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing 'model_gguf-unsloth.f16.gguf' to 'model_gguf_q4_k_m-unsloth.Q4_k_m.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from model_gguf-unsloth.f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_coun

In [62]:
!ls -lh model_gguf_q4_k_m-unsloth.Q4_k_m.gguf

-rw-r--r-- 1 root root 4.6G Apr 19 18:37 model_gguf_q4_k_m-unsloth.Q4_k_m.gguf


In [63]:
!huggingface-cli upload --repo-type model weege007/llama-3-8b-bnb-4bit-alpaca-merged-16bit model_gguf_q4_k_m-unsloth.Q4_k_m.gguf

Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
model_gguf_q4_k_m-unsloth.Q4_k_m.gguf: 100% 4.92G/4.92G [02:11<00:00, 37.3MB/s]
https://huggingface.co/weege007/llama-3-8b-bnb-4bit-alpaca-merged-16bit/blob/main/model_gguf_q4_k_m-unsloth.Q4_k_m.gguf


In [64]:
!./llama.cpp/main -ngl 33 -c 0 -e \
  -p '<|start_header_id|>user<|end_header_id|>\n\nWhat is the capital of France?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n' \
  -r '<|eot_id|>' \
  -m model_gguf_q4_k_m-unsloth.Q4_k_m.gguf \
  && echo "The capital of France is Paris."


Log start
main: build = 2698 (637e9a86)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1713552045
llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from model_gguf_q4_k_m-unsloth.Q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:  

In [ ]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model_gguf_q8_0", tokenizer, quantization_method = "q8_0",)
if False: model.push_to_hub_gguf("weege007/llama-3-8b-bnb-4bit-alpaca-merged-16bit", tokenizer, token = token)
# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model_gguf_f16", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("weege007/llama-3-8b-bnb-4bit-alpaca-merged-16bit", tokenizer, quantization_method = "f16", token = token)
# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model_gguf_q4_k_m", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("weege007/llama-3-8b-bnb-4bit-alpaca-merged-16bit", tokenizer, quantization_method = "q4_k_m", token = token)

现在，在`llama.cpp`中使用`model-unsloth.gguf`文件或`model-unsloth-Q4_K_M.gguf`文件，或者使用像`GPT4All`这样的基于UI的系统。您可以通过[这里](https://gpt4all.io/index.html)安装GPT4All。

完成了！如果您对Unsloth有任何疑问，我们有一个[Discord](https://discord.gg/u54VK8m8tk)频道！如果您发现任何错误，或者想要了解最新的LLM内容，或者需要帮助，加入项目等等，请随时加入我们的Discord！

一些其他链接：
1. Zephyr DPO 2倍速 [免费Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2倍速 [免费Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4倍速完整Alpaca 52K在1小时内 [免费Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2倍速 [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [免费Kaggle版本](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. 我们还和🤗 HuggingFace一起做了一个[博客](https://huggingface.co/blog/unsloth-trl)，我们在TRL的[文档](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)中也有介绍！
7. `ChatML`用于ShareGPT数据集，[对话笔记本](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. 类似写小说的文本补全[笔记本](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> 如果可以，请支持我们的工作！谢谢！
</div>